<a href="https://colab.research.google.com/github/tgstoecker/teaching/blob/master/AppliedBioinformatics/Notebooks/GeneCountsVenn2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Venn Diagrammes - the power of the theory

##Installation of necessary R packages

In [0]:
R.Version()

install.packages("BiocManager", verbose = TRUE)

BiocManager::install(version = "3.9", ask = FALSE)

# install the VennDiagramme package - you need to do this only once
install.packages("VennDiagram")

if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("limma", version = "3.9")

if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("edgeR", version = "3.9")

## Load all required packages


In [0]:
library(limma)
library(edgeR)
library(VennDiagram)

##Quick & easy data handing

Read in the results - the count matrix of ALL 4 maize lines:

In [0]:
# row.names = 1 results in GeneIDs as rownames
counts <- "https://raw.githubusercontent.com/tgstoecker/teaching/master/AppliedBioinformatics/gene-level_ALL/total_file.count"
fc_res_all <- read.table(counts, header = T, row.names = 1)

Check the names of your columns:


In [0]:
colnames(fc_res_all)

Shorten the column names indicating the samples - e.g.:


In [0]:
colnames(fc_res_all) <- sub("_trimmed_sorted.bam", "", colnames(fc_res_all))

Create a vector indicating treatment conditions of the samples -  
logic: columns left to right


In [0]:
group_all = c("B73_control", "B73_control", "B73_control", "B73_control", "B73_drought", "B73_drought", "B73_drought", "B73_drought",
              "Mo17_control", "Mo17_control", "Mo17_control", "Mo17_control", "Mo17_drought", "Mo17_drought", "Mo17_drought", "Mo17_drought",
              "MxB_control", "MxB_control", "MxB_control", "MxB_control", "MxB_drought", "MxB_drought", "MxB_drought", "MxB_drought",
              "BxM_control", "BxM_control", "BxM_control", "BxM_control", "BxM_drought", "BxM_drought", "BxM_drought", "BxM_drought")
group_all

Create a DGE list object:

In [0]:
#For our purposes the DGEList-object should contain matrixes/dataframes of raw counts, group/treatment info as well as gene names 
dge_all = DGEList(counts = fc_res_all[, 6:37], group = group_all, genes = rownames(fc_res_all))

Creating the design matrix:


In [0]:
design_all <- model.matrix(~0+group_all)
design_all

In [0]:
# check the column names - these should be your groups of interest
colnames(design_all)
#let's improve that a bit
colnames(design_all) <- sub("group_all", "", colnames(design_all))
colnames(design_all)

Perform the filtering of the DGEList-object:


In [0]:
keep <- filterByExpr(dge_all, design_all)
dge_all <- dge_all[keep, , keep.lib.sizes=FALSE]
dim(dge_all)

Perform normalization:

In [0]:
dge_all <- calcNormFactors(dge_all, method = "TMM")

##Quick foray back to MDS plots - this time in useful

Let's check out a MDS plot, since this time around the dataset is much more reasonable for such a visualization.  
We should get a plot very similar to Figure 1A of the Hochholdinger publication - compare:

In [0]:
pch <- c(0, 3, 7, 11, 14, 17, 21, 25, 29, 33)
colors <- rep(c("blue", "red", "green", "black", "violet", "yellow", "brown", "grey"))
plotMDS(dge_all, col = colors[dge_all$samples$group], pch = pch[dge_all$samples$group])
legend("topright", legend=levels(dge_all$samples$group), pch=pch, col=colors, ncol=1)

##Venn diagrammes 

Before we start we first choose a set of genes which pass an expression threshold (shown here is quick and dirty example good enough for demonstration purposes).


1. Manual transformation of our DGEList object into log scale CPM values


In [0]:
lcpm <- cpm(dge_all, log=TRUE, normalized.lib.sizes =TRUE)

2. Let's create a reasonable lcpm cutoff


In [0]:
L <- mean(dge_all$samples$lib.size) * 1e-6
M <- median(dge_all$samples$lib.size) * 1e-6
c(L, M)
lcpm.cutoff <- log2(10/M + 2/L)
summary(lcpm)
colnames(lcpm)

3. For each of our 4 lines and conditions we now extract the corresponding columns, calculate means per row and retain those over the threshold -> also, check your colnames/the corresponding index number

In [0]:
#control treatment data
B73_set.control <- lcpm[, 1:4]
B73_genes_ov_cutoff.control <- rownames(data.frame(subset(B73_set.control, rowMeans(B73_set.control) > lcpm.cutoff)))

Mo17_set.control <- lcpm[, 9:12]
Mo17_genes_ov_cutoff.control <- rownames(data.frame(subset(Mo17_set.control, rowMeans(Mo17_set.control) > lcpm.cutoff)))

MxB_set.control <- lcpm[, 17:20]
MxB_genes_ov_cutoff.control <- rownames(data.frame(subset(MxB_set.control, rowMeans(MxB_set.control) > lcpm.cutoff)))

BxM_set.control <- lcpm[, 25:28]
BxM_genes_ov_cutoff.control <- rownames(data.frame(subset(BxM_set.control, rowMeans(BxM_set.control) > lcpm.cutoff)))

###
##drought treatment data
B73_set.drought <- lcpm[, 5:8]
B73_genes_ov_cutoff.drought <- rownames(data.frame(subset(B73_set.drought, rowMeans(B73_set.drought) > lcpm.cutoff)))

Mo17_set.drought <- lcpm[, 13:16]
Mo17_genes_ov_cutoff.drought <- rownames(data.frame(subset(Mo17_set.drought, rowMeans(Mo17_set.drought) > lcpm.cutoff)))

MxB_set.drought <- lcpm[, 21:24]
MxB_genes_ov_cutoff.drought <- rownames(data.frame(subset(MxB_set.drought, rowMeans(MxB_set.drought) > lcpm.cutoff)))

BxM_set.drought <- lcpm[, 29:32]
BxM_genes_ov_cutoff.drought <- rownames(data.frame(subset(BxM_set.drought, rowMeans(BxM_set.drought) > lcpm.cutoff)))

*The final goal will be to create a four way venn diagramme of all expressed genes in the control samples*

But first let's draw venn diagram of the genes expressed in B73 control vs. B73 drought:

In [0]:
# build the intersection
B73_control_drought <- intersect(B73_genes_ov_cutoff.control, B73_genes_ov_cutoff.drought) # build the intersection
draw.pairwise.venn(area1=length(B73_genes_ov_cutoff.control), area2=length(B73_genes_ov_cutoff.drought), cross.area=length(B73_control_drought))

In [0]:
# draw venn diagram of the genes expressed in B73 control vs. Mo17 control
# build the intersection
B73_Mo17_control <- intersect(B73_genes_ov_cutoff.control, Mo17_genes_ov_cutoff.drought) 
draw.pairwise.venn(area1=length(B73_genes_ov_cutoff.control), area2=length(Mo17_genes_ov_cutoff.control), cross.area=length(B73_Mo17_control))

In [0]:
### Now we want to draw a FOUR way venn diagram of the control samples of all four genotypes
# have a look at the command and options
?draw.quad.venn


#### Step 1: build pair wise intersections of the control samples ####
B73_Mo17_intersect <- intersect(B73_genes_ov_cutoff.control, Mo17_genes_ov_cutoff.drought)



#### add the commands to build the other 5 pair wise intersections of the control samples ####



#### Step 2: build tripple intersections ####
B73_Mo17_MxB_intersect <- intersect(B73_Mo17_intersect, MxB_genes_ov_cutoff.control)




#### add the commands to build the other 3 tripple intersections of the control samples ####



#### Step 4: Add the command to build the one four way intersection of the control samples ####

#### use the 'draw.quad.venn' function from the 'VennDiagram' package to plot a four way venn diagram ####
#### use the 'category' option to label the sets appropriately ####
#### use the 'col' and/or 'fill' options to colour the sets ####
draw.quad.venn()





#BONUS: 

If you have time:
1. Plot a 4 way venn diagramme for drought treatment  

2. Use the 'union' function to generate a list of all gene expressed in any of the four samples  

3. Use the 'setdiff' function to generate lists of the genes exclusively expressed in each sample:  
- only B73  
- only Mo17  
- only MxB  
- only BxM  